In [2]:
import requests
import json
import pandas as pd

In [7]:
crime_type_map_dict={
0:'Violence and sexual offences',
1:'Anti-social behaviour',
2:'Public order',
3:'Criminal damage and arson',
4:'Other theft',
5:'Vehicle crime',
6:'Shoplifting',
7:'Burglary',
8:'Drugs',
9:'Other crime',
10:'Bicycle theft',
11:'Theft from the person',
12:'Robbery',
13:'Possession of weapons'
}

True

In [8]:
%%time 
crime_points_req = requests.get(
    f'http://172.30.80.224:8000/api/crime/crime_points',
    {"lat": 51.37570431724386, "lon":0.09920879452076632, "radius":1, "lad_cd":"E09000006"}
)
crime_points_dict = [json.loads(x) for x in json.loads(crime_points_req.text)]
# print(crime_points_dict)
crime_pdf = pd.DataFrame.from_dict(crime_points_dict)
crime_pdf["crime_type"]=crime_pdf["crime_type_cd"].map(crime_type_map_dict)

CPU times: user 12.8 ms, sys: 0 ns, total: 12.8 ms
Wall time: 477 ms


In [9]:
crime_pdf.count()

mnth             705
crime_type_cd    705
lat              705
lon              705
postcode         705
lad_cd           705
distance_        705
radius           705
tooltip          705
crime_type       705
dtype: int64

In [10]:
crime_pdf.to_pickle("./toyData.pkl")

In [11]:
%%time 
crime_pdf=pd.read_pickle("./toyData.pkl")

CPU times: user 4.15 ms, sys: 0 ns, total: 4.15 ms
Wall time: 2.37 ms


In [12]:
crime_pdf.count()

mnth             705
crime_type_cd    705
lat              705
lon              705
postcode         705
lad_cd           705
distance_        705
radius           705
tooltip          705
crime_type       705
dtype: int64

# LocateControl

In [69]:
import dash_leaflet as dl
from dash_extensions.enrich import html, DashProxy
import dash_leaflet.express as dlx
from jupyter_dash import JupyterDash
from dash_extensions.javascript import assign

from dash import dcc, callback, Input, Output
from time import sleep
# polygon = dl.Polygon(positions=[
#     [50.72485912389739,-3.533088805791474], 
#     [50.72483686731482, -3.5333147806665437], 
#     [50.724770952875105, -3.5335320714538927]
# ])

app=JupyterDash()

user_click_icon ={
    "iconUrl": "https://raw.githubusercontent.com/10hith/excelanalysis/release/aio/app/dash_apps/assets/point_select.ico",
    "iconSize": [20, 30],
    "iconAnchor": [20, 20],
    "popupAnchor": [-10, -50],
    "shadowSize": [41, 41]
    }


# 40, 40 works; but no other size works
red_icon_func = assign("""function(feature, latlng){
const flag = L.icon({iconUrl: `https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-red.png`, iconSize: [20,30]});
return L.marker(latlng, {icon: flag});
}""")

gold_icon_func = assign("""function(feature, latlng){
const flag = L.icon({iconUrl: `https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-gold.png`, iconSize: [20,30]});
return L.marker(latlng, {icon: flag});
}""")

# https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-red.png

# blue_icon ={
#     "iconUrl": "https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-blue.png",
#     "iconSize": [40, 40],
#     "iconAnchor": [20, 20],
#     "popupAnchor": [-10, -50],
#     "shadowSize": [41, 41]
#     }

# selected_point=dl.Marker(
#         position=[click_lat_lng[0], click_lat_lng[1]], children=dl.Tooltip(f"{click_lat_lng}"), icon=user_click_icon
#         )

big_gree_icon='https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-green.png'

crime_type_0=[dl.GeoJSON(
    data=dlx.dicts_to_geojson(crime_pdf.query("crime_type_cd==0").to_dict("records")),
    cluster=True,
    spiderfyOnMaxZoom=True,
    options=dict(pointToLayer=gold_icon_func)
)]
crime_type_1=[dl.GeoJSON(
    data=dlx.dicts_to_geojson(crime_pdf.query("crime_type_cd==1").to_dict("records")),
    cluster=True,
    options=dict(pointToLayer=red_icon_func)
)]
                    
app.layout = html.Div([
    
    dl.Map([
        dl.TileLayer(),
        m_selectedPoint := dl.LayerGroup(),
        m_circleMarker := dl.LayerGroup(),
        dl.LayersControl([
                dl.Overlay(dl.LayerGroup(crime_type_0), name="type_0", checked=True),
                dl.Overlay(dl.LayerGroup(crime_type_1), name="type_1", checked=False)
        ])
        ], 
        zoom=7, id="map", center=(51.35421253175895,0.0960463563716222), style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}
    ),
    _selectedPoint:=dcc.Store(id="_selectedPoint", data=[]),
    debugDiv:=html.Div(id="debugDiv", children="Hi there MFucka"),
])

@callback(
    [
        Output("_selectedPoint", "data")
    ],
    [Input("map", "click_lat_lng")],
    prevent_initial_call=True
    )
def store_selected_point(click_lat_lng):
    lat, lon = click_lat_lng[0], click_lat_lng[1]
    return [{"lat": lat, "lon": lon}]

@callback(
    [
        Output(m_selectedPoint, "children")
    ],
    [Input("_selectedPoint", "data")],
    prevent_initial_call=True
    )
def display_point_on_select(point_dict):
    lat=point_dict["lat"]
    lon=point_dict["lon"]
    point_marker = dl.Marker(
        position=[lat,lon], children=dl.Tooltip(f"{lat,lon}"), icon=user_click_icon
        )
    return [point_marker]


@callback(
    [
        Output(m_circleMarker, "children")
    ],
    [Input("_selectedPoint", "data")],
    prevent_initial_call=True
    )
def display_point_on_select(point_dict):
    lat=point_dict["lat"]
    lon=point_dict["lon"]
    sleep(1)
    circle_pointer=dl.Circle(center=[lat, lon],radius=322,children=dl.Tooltip("0.2 Mile radius"))
    return [circle_pointer]


if __name__ == '__main__':
    app.run_server(dev_tools_hot_reload=True, mode='external')

Dash app running on http://127.0.0.1:8050/


# client side filter of dcc.store

In [13]:
import pandas as pd 

crime_pdf=pd.read_pickle("./toyData.pkl")
crime_pdf.dtypes

mnth              object
crime_type_cd      int64
lat              float64
lon              float64
postcode          object
lad_cd            object
distance_        float64
radius            object
tooltip           object
crime_type        object
dtype: object

In [15]:
crime_pdf["mnth"].unique()

array(['2021-10-01', '2021-11-01', '2021-12-01'], dtype=object)

# Got Danfo working :-) 

In [111]:
import dash_leaflet as dl
from dash_extensions.enrich import html, DashProxy
import dash_leaflet.express as dlx
from jupyter_dash import JupyterDash
from dash_extensions.javascript import assign

from dash import dcc, callback, Input, Output, State
from time import sleep
# polygon = dl.Polygon(positions=[
#     [50.72485912389739,-3.533088805791474], 
#     [50.72483686731482, -3.5333147806665437], 
#     [50.724770952875105, -3.5335320714538927]
# ])

# external_stylesheets="https://cdn.jsdelivr.net/npm/danfojs@1.0.2/lib/bundle.min.js"

app=JupyterDash()

# https://cdn.jsdelivr.net/npm/danfojs@1.0.2/lib/bundle.min.js

app.layout = html.Div([
    debugDiv:=html.Div(),
    html.P([
        '''Data Profiling is the process of collecting statistics or informative summaries about that data.'''
    ], id="htmlP"),
    html.Div(id="debugDiv1"),
    html.Div(id="debugDiv2"),
    dcc.Dropdown(id="mnthDropDown"),
    html.Div(id="plot_div"),
    html.Div(id="hist_div"),
    dcc.Store(id="myStore",data=crime_pdf.to_dict("records")),
    dcc.Store(id="jsOutput"),
    html.Button(id='Submit', n_clicks=0),
])

# @callback(Output(debugDiv, "children"), 
#           Input("myStore", "data")
#          )
# def filter_callback(data):
#     return [data[0]["website"]]



# app.clientside_callback(
#     """
#     function(n_clicks,data) {
#     var filter_data=data.filter(
#         function(item){return item.radius==".2" && [0].includes(item.crime_type_cd) && ["2021-10-01","2021-12-01"].includes(item.mnth)}
#         );
#     console.log(filter_data);
#     return JSON.stringify(filter_data);
#     }
#     """,
#     Output('debugDiv1', 'children'),
#     Input("Submit", "n_clicks"),
#     State("myStore", "data"),
#     prevent_initial_call=True
# )

# app.clientside_callback(
#     """
#     function(n_clicks, data) {
#     let df = new dfd.DataFrame(data)
    
#     df.plot("plot_div").table();
#     df['crime_type_cd'].plot("hist_div").hist();
#     let mnthArry=df['mnth'].unique()
#     console.log(mnthArry)
#     return [JSON.stringify(df)], mnthArry
#     }
#     """,
#     [
#         Output('debugDiv1', 'children'),
#         Output("mnthDropDown","options")
#     ],
#     Input("Submit", "n_clicks"),
#     State("myStore", "data"),
#     prevent_initial_call=True
# )

# # Dropdown works when you pass a list
# app.clientside_callback(
#     """
#     function(n_clicks, data) {
#     let df = new dfd.DataFrame(data)
    
#     df.plot("plot_div").table();
#     df['crime_type_cd'].plot("hist_div").hist();
#     let mnthArry=df["mnth"].unique()
#     console.log("The data type of mnthArry is =>")
#     console.log(typeof df);
#     return [['wow', 'this', 'worked', 'before']];
#     }
#     """,
#     [
#         Output('mnthDropDown', 'options')
#     ],
#     Input("Submit", "n_clicks"),
#     State("myStore", "data")
# )

# Dropdown works when you pass a list
app.clientside_callback(
    """
    function(n_clicks, data) {
    return [
        Array.from(new Set(data.map((x)=>x.mnth))), 
        Object.values(data).length
        ];
    }
    """,
    [
        Output('mnthDropDown', 'options'),
        Output('debugDiv2', 'children')
    ],
    Input("Submit", "n_clicks"),
    State("myStore", "data"),
    prevent_initial_call=True
)


# @callback(Output("debugDiv1", "children"), 
#           Input("jsOutput", "data")
#          )
# def display_js_output(data):
#     return [str(data)]


if __name__ == '__main__':
    app.run_server(dev_tools_hot_reload=True, port=8000,mode='external')

Dash app running on http://127.0.0.1:8000/
